In [6]:
import json
import re
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.colorbar import contour
%matplotlib inline
import cv2
print(cv2.__version__)

4.7.0


In [ ]:
#### df 뽑아내는 함수 (wh -> 장단축으로 변환)
def find_contours_ls(img, threshold_percent):
    '''
    입력된 Image에서 Contous를 추출하여 max_area, long, short, ratio를 추출하는 함수
    각 Image에서 최대 Pixel을 기준으로 입력받은 Threshold percent만큼 임계값을 설정함

    예외처리)
    입력된 Image가 읽혀지지 않으면, Output모두 'Img_error'로 출력됨
    입력된 Image에서 threshold값으로 추출되는 Contous가 없는 경우, Output 모두 'None'으로 출력됨

    Input
     - img : 이미지의 경로
     - threshold_percent(float) : 임계값설정 % (0.nn 으로 입력)

     Output
     - max_area(float) : 찾은 Contous의 면적 (Contous가 여러개일 경우 최대값으로 설정) 
     - long(float, rount3) : Contous의 장축
     - short(float, round3) : Contous의 단축
     - ratio(float, round3) : Contous의 종횡비 (장축/단축)
    '''
    
    try:
        img_bgr = cv2.imread(img)
        img_bitwise_not_bgr = cv2.bitwise_not(img_bgr)
        img_bitwise_not_bgr2gray = cv2.cvtColor(img_bitwise_not_bgr, cv2.COLOR_BGR2GRAY)
        max_pixel = img_bitwise_not_bgr2gray.max()
        threshold_value = max_pixel * threshold_percent 
        _, img_binary = cv2.threshold(img_bitwise_not_bgr2gray, threshold_value,max_pixel,1)
        contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cont_dict = dict()

        if len(contours) > 1: # contour 1개 이상
            for i in range(len(contours)):
                tmp_area = cv2.contourArea(contours[i])
                cont_dict[i] = tmp_area
            max_key = max(cont_dict, key=cont_dict.get)

        elif len(contours) == 1:
            max_key = 0
        else: # 잡힌 contour 없음
            max_key = None
        
        # 설정된 max_key 값으로 contour data 추출
        if max_key != None:
            _, wh, _ = cv2.minAreaRect(contours[max_key])
            long = round(max(wh),3)
            short = round(min(wh),3)
            max_area = float(cv2.contourArea(contours[max_key]))
            ratio =  round(float(long/short),3)
            contours = contours[max_key]
        else:
            max_area = long = short = ratio = 'None'

    # Img가 깨져서 cv2.imread(img)가 안되는 경우
    except Exception as e: 
        # print(img.split('/')[-1], ' error msg: ',e)
        max_area = long = short = ratio = 'Img_error'

    return max_area, long, short, ratio

In [24]:
# ### Try
# img = "C:/Users/Chaaaaabbo/Desktop/0061_63.bmp"
# threshold_percent = 0.1

# try:
#     img_bgr = cv2.imread(img)
#     img_bitwise_not_bgr = cv2.bitwise_not(img_bgr)
#     img_bitwise_not_bgr2gray = cv2.cvtColor(img_bitwise_not_bgr, cv2.COLOR_BGR2GRAY)
#     max_pixel = img_bitwise_not_bgr2gray.max()
#     threshold_value = max_pixel * threshold_percent 
#     _, img_binary = cv2.threshold(img_bitwise_not_bgr2gray, threshold_value,max_pixel,1)
#     contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     cont_dict = dict()

#     if len(contours) > 1: # contour 1개 이상
#         for i in range(len(contours)):
#             tmp_area = cv2.contourArea(contours[i])
#             cont_dict[i] = tmp_area
#         max_key = max(cont_dict, key=cont_dict.get)

#     elif len(contours) == 1:
#         max_key = 0
#     else: # 잡힌 contour 없음
#         max_key = None
    
#     # 설정된 max_key 값으로 contour data 추출
#     if max_key != None:
#         _, wh, _ = cv2.minAreaRect(contours[max_key])
#         long = round(max(wh),3)
#         short = round(min(wh),3)
#         max_area = float(cv2.contourArea(contours[max_key]))
#         ratio =  round(float(long/short),3)
#         contours = contours[max_key]
#     else:
#         max_area = long = short = ratio = 'None'
# except Exception as e:
#     print(img.split('/')[-1], ' error msg: ',e)
#         # max_area , w , h , ratio = None
#     max_area = long = short = ratio = 'Img_error'

# print(max_area, long, short, ratio)

746.0 59.465 17.366 3.424


In [ ]:
import os
import pandas as pd
import warnings
import time
warnings.filterwarnings('ignore')


root_dir = 'E:\\image'
# os.makedirs(img_save_dir,exist_ok=True)
samples = [f for f in os.listdir(root_dir)]

# 변경 
columns= ['specimen_no', 'Layer No.','contours_threshold', 'area', 'long', 'short', 'aspect_ratio']

threshold_percent = 0.1

# 일부만 사용시 제외할 시편no. 입력
finished = []

print(samples)

for sample in samples:
    sample_num = sample.split('_')[0]
    print(sample_num)

    if sample_num in finished:
        continue

    sample_dir = root_dir+'\\'+sample+'\\On_Axis_Images'
    temp_files =  [temp_file for temp_file in os.listdir(sample_dir) if temp_file.endswith('.bmp')]
    # print('='*20,'진행중인 시편', sample_num,'='*20)
    array_list = [[0] for i in range(len(temp_files))]
    # print(sample_num,'번 시편 길이 : ',len(temp_files))

    for idx, temp_file in enumerate(temp_files):
        temp_file_path = sample_dir+'\\'+temp_file

        try:
            max_area, long, short, ratio = find_contours_ls(temp_file_path,threshold_percent)
            data = [sample_num,temp_file,threshold_percent,str(max_area), str(long),str(short), str(ratio)]
            # temp_df = temp_df.append(pd.Series(data, index=temp_df.columns), ignore_index=True) -> 속도저하로 삭제
            array_list[idx] = data
        except Exception as e:
            print('error msg: ',e)

    if idx % 5000 == 0:
        print(f'{idx}개 완료')

    array_df = pd.DataFrame(array_list, columns = columns)
    array_df.to_csv(f'E:\\workflow\\Cha\\on_bbox_info\\0_img_info{sample_num}_thres_10_ls.csv',mode ='w', header=True, encoding='utf-8-sig')